# Initial step: load the dataset (Benchmark3)

A common dataset generated in the context of second benchmark will be used to evaluate the augmented simulators. This initial step aims at loading it once and for all.

In [1]:
import os
from pprint import pprint
from lips.neurips_benchmark import NeuripsBenchmark3
path_benchmark = os.path.join("reference_data")
neurips_benchmark3 = NeuripsBenchmark3(path_benchmark=path_benchmark,
                                       load_data_set=True)

# Training an augmented simulator 
In this benchmark the augmented simulators will predict all the power flow related variables (active $p$ and reactive $q$ powers, voltages $v$ and currents $a$) alongside the voltage angles ($\theta$) contrary to the first and second benchmarks which were concentrated only on power flow variables.

## Fully connected architecture
In this section we explain how to tune an available model. We take the example of the `FullyConnectedAS` that is an available fully connected neural network.

The first step is to create the class you want to use, with the meta parameters you want to test. For this example: 

In [2]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import FullyConnectedAS

# the three lines bellow might be familiar to the tensorflow users. They tell tensorflow to not take all
# the GPU video RAM for the model.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for el in physical_devices:
    tf.config.experimental.set_memory_growth(el, True)

simulator_FC = FullyConnectedAS(name="test_FullyConnectedAS_benchmark3",
                                # `attr_x` represents the variables of the dataset you want to use to predict 
                                # the output.
                                attr_x=("prod_p", "prod_v", "load_p", "load_q", "line_status", "topo_vect"),
                                # `attr_y` represents the variables of the dataset you want to predict
                                # flow variables alongside the angles are added
                                attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "load_v", "prod_q", "theta_or", "theta_ex"),
                                # `sizes_layer` represents the size of each hidden layer in the neural network. The number
                                # of layers is determined by the length of this list, for example
                                sizes_layer=(300, 300, 300, 300),
                                # `lr` is the learning rate
                                lr=3e-4, 
                                # `layer` is the type of keras layer you want to use. We don't recommend to 
                                # change it
                                layer=Dense,
                                # `layer_act` is the activation function you want to use after each layer
                                layer_act="relu",
                                # `loss` is the training loss
                                loss="mse",  # loss used to train the model
                                # `batch_size` is the size of the batch for training
                                batch_size=128)

In [3]:
simulator_FC.train(nb_iter=200,
                   train_dataset=neurips_benchmark3.train_dataset,
                   val_dataset=neurips_benchmark3.val_dataset)

Epoch 1/200
782/782 [==============================] - 4s 5ms/step - loss: 0.2801 - val_loss: 0.0166
Epoch 2/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0115 - val_loss: 0.0081
Epoch 3/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0062 - val_loss: 0.0058
Epoch 4/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0044 - val_loss: 0.0045
Epoch 5/200
782/782 [==============================] - 4s 5ms/step - loss: 0.0034 - val_loss: 0.0040
Epoch 6/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0028 - val_loss: 0.0032
Epoch 7/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0024 - val_loss: 0.0026
Epoch 8/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 9/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 10/200
782/782 [==============================] - 3s 4ms/step - loss: 0.0016 - val_lo

782/782 [==============================] - 3s 4ms/step - loss: 3.5739e-04 - val_loss: 3.5092e-04
Epoch 78/200
782/782 [==============================] - 3s 4ms/step - loss: 5.8855e-04 - val_loss: 3.6860e-04
Epoch 79/200
782/782 [==============================] - 3s 4ms/step - loss: 3.5022e-04 - val_loss: 5.9229e-04
Epoch 80/200
782/782 [==============================] - 3s 4ms/step - loss: 3.4466e-04 - val_loss: 7.4453e-04
Epoch 81/200
782/782 [==============================] - 3s 4ms/step - loss: 4.5856e-04 - val_loss: 4.6692e-04
Epoch 82/200
782/782 [==============================] - 3s 4ms/step - loss: 3.5001e-04 - val_loss: 6.3207e-04
Epoch 83/200
782/782 [==============================] - 3s 4ms/step - loss: 3.6265e-04 - val_loss: 4.1316e-04
Epoch 84/200
782/782 [==============================] - 3s 4ms/step - loss: 3.4618e-04 - val_loss: 3.0900e-04
Epoch 85/200
782/782 [==============================] - 3s 4ms/step - loss: 3.2092e-04 - val_loss: 3.7666e-04
Epoch 86/200
782/782 [=

save it

In [4]:
simulator_FC.save(path_save)
simulator_FC.save_metadata(path_save)

load it

In [23]:
path_trained_models = os.path.join("trained_models")
from tensorflow.keras.layers import Dense
from lips.augmented_simulators import FullyConnectedAS

# recreate the baseline
simulator_FC = FullyConnectedAS(name="test_FullyConnectedAS_benchmark3",
                                attr_x=("prod_p", "prod_v", "load_p", "load_q", "line_status", "topo_vect"),
                                attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "load_v", "prod_q", "theta_or", "theta_ex"),
                                sizes_layer=(300, 300, 300, 300),
                                lr=3e-4, 
                                layer=Dense,
                                layer_act="relu",
                                loss="mse",  # loss used to train the model
                                batch_size=128)
# TODO create a wrapper for these 3 calls
simulator_FC.load_metadata(path_trained_models)
simulator_FC.init()
simulator_FC.restore(path_trained_models)

# Evaluate the augmented simulator 

In [24]:
fc_metrics_per_dataset = neurips_benchmark3.evaluate_augmented_simulator(simulator_FC)

A log file including some verifications is created at root directory with the name logs.log


C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add

In [26]:
neurips_benchmark3.predictions.keys()

dict_keys(['val', 'test', 'test_ood_topo'])

In [25]:
neurips_benchmark3.predictions["test"].keys()

dict_keys(['a_or', 'a_ex', 'p_or', 'p_ex', 'q_ex', 'q_or', 'v_or', 'v_ex', 'load_v', 'prod_q', 'theta_or', 'theta_ex'])

In [12]:
neurips_benchmark3._test_dataset.data.keys()

dict_keys(['prod_p', 'prod_v', 'load_p', 'load_q', 'line_status', 'topo_vect', 'a_or', 'a_ex', 'p_or', 'p_ex', 'q_or', 'q_ex', 'prod_q', 'load_v', 'v_or', 'v_ex', 'theta_or', 'theta_ex', 'load_theta', 'gen_theta', 'storage_theta'])

In [13]:
neurips_benchmark3._test_dataset.data.keys() - neurips_benchmark3.predictions["test"].keys()

{'gen_theta',
 'line_status',
 'load_p',
 'load_q',
 'load_theta',
 'prod_p',
 'prod_v',
 'storage_theta',
 'topo_vect'}

In [16]:
import numpy as np

In [21]:
np.save("ref_obs", neurips_benchmark3._test_dataset.data)

In [22]:
np.save("predictions", neurips_benchmark3.predictions["test"])

In [28]:
import pickle

a_file = open("predictions_FC_test.pkl", "wb")
pickle.dump(neurips_benchmark3.predictions["test"], a_file)
a_file.close()

a_file = open("predictions_FC_ood.pkl", "wb")
pickle.dump(neurips_benchmark3.predictions["test_ood_topo"], a_file)
a_file.close()

### ML Metrics 

In [4]:
ML_metrics = 0
print("MAPE90")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["mape90"])
print("MAPE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["mape_avg"])
print("MAE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.00453048923659429,
 'a_or': 0.004571197769038052,
 'p_ex': 0.005380266589622638,
 'p_or': 0.005441060604053268,
 'q_ex': 0.010820239959033444,
 'q_or': 0.012243016250860288,
 'theta_ex': 0.00594510889206132,
 'theta_or': nan,
 'v_ex': 0.0020258985388451065,
 'v_or': 0.002116077964796591}
MAPE
{'a_ex': 0.01206509496029071,
 'a_or': 0.012328329055745853,
 'load_v': 0.00014130105748821816,
 'p_ex': 0.012246075987036283,
 'p_or': 0.0122042014707874,
 'prod_q': 0.005787441796298978,
 'q_ex': 0.014143698071288905,
 'q_or': 0.012699580187556642,
 'theta_ex': 0.006121490332365712,
 'theta_or': nan,
 'v_ex': 0.0018207928324632107,
 'v_or': 0.0018710782328293402}
MAE
{'a_ex': 2.4359219074249268,
 'a_or': 1.7547935247421265,
 'load_v': 0.007250834722071886,
 'p_ex': 0.1440560519695282,
 'p_or': 0.1459747850894928,
 'prod_q': 0.13227196037769318,
 'q_ex': 0.06623383611440659,
 'q_or': 0.06187693402171135,
 'theta_ex': 0.044860786692496345,
 'theta_or': 0.040842502893081604,
 'v_e

### OOD Generalization 

In [7]:
print("MAPE90")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape90"])
print("MAPE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape_avg"])
print("MAE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.20107159106043712,
 'a_or': 0.2010885507612319,
 'p_ex': 0.23931476475031105,
 'p_or': 0.241871617038327,
 'q_ex': 0.34331784958160283,
 'q_or': 0.36442009321746427,
 'theta_ex': 0.15957073813573144,
 'theta_or': nan,
 'v_ex': 0.03118037104864476,
 'v_or': 0.030783094899299146}
MAPE
{'a_ex': 0.21175651492018047,
 'a_or': 0.2150390137056632,
 'p_ex': 0.2739599888702738,
 'p_or': 0.2756300244671787,
 'q_ex': 0.2712641773304669,
 'q_or': 0.2659483474492036,
 'theta_ex': 0.1304486416400271,
 'theta_or': nan,
 'v_ex': 0.027558933854545786,
 'v_or': 0.027649661776876232}
MAE
{'a_ex': 65.6705551147461,
 'a_or': 45.574684143066406,
 'p_ex': 3.94745135307312,
 'p_or': 4.019582748413086,
 'q_ex': 1.6291145086288452,
 'q_or': 1.5206332206726074,
 'theta_ex': 1.7184120090077706,
 'theta_or': 1.6142652562770508,
 'v_ex': 2.8946361541748047,
 'v_or': 3.7403595447540283}
NRMSE
{'a_ex': 0.056359924376010895,
 'a_or': 0.056323640048503876,
 'p_ex': 0.050281137228012085,
 'p_or': 0.050

### Physics compliance 

In [5]:
print("current pos")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["line_status"])
print("KCL")
print("Violation percentage: ", fc_metrics_per_dataset["test"][1]["KCL"]["violation_percentage"])

current pos
0.035045
0.03378
voltage pos
0.031935
0.03212
loss
0.220215
line_status
{'a_ex_not_null': 11860.0,
 'a_or_not_null': 11860.0,
 'a_violations': 1.0,
 'p_ex_not_null': 11860.0,
 'p_or_not_null': 11860,
 'p_violations': 1.0,
 'q_ex_not_null': 11860.0,
 'q_or_not_null': 11860.0,
 'q_violations': 1.0}
KCL
Violation percentage:  93.5364864864865


In [6]:
fc_metrics_per_dataset["test"][1]["KCL_new"]

{'violation_prop_obs_level': 1.0,
 'violation_prop_sub_level': 0.4843142857142857}

In [10]:
print("current pos")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["line_status"])
#pprint(dc_metrics_per_dataset["test_ood_topo"][1])
print("KCL")
print("Violation percentage: ", fc_metrics_per_dataset["test_ood_topo"][1]["KCL"]["violation_percentage"])

current pos
0.02754
0.02741
voltage pos
0.01226
0.012315
loss
0.362545
line_status
{'a_ex_not_null': 21843.0,
 'a_or_not_null': 21843.0,
 'a_violations': 1.0,
 'p_ex_not_null': 21843.0,
 'p_or_not_null': 21843,
 'p_violations': 1.0,
 'q_ex_not_null': 21843.0,
 'q_or_not_null': 21843.0,
 'q_violations': 1.0}
KCL
Violation percentage:  99.11331916448944


In [7]:
fc_metrics_per_dataset["test_ood_topo"][1]["KCL_new"]

{'violation_prop_obs_level': 1.0,
 'violation_prop_sub_level': 0.48201428571428573}

# LeapNets 

In [10]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import LeapNetAS

In [15]:
leapNet = LeapNetAS(name="test_leapNetAS_benchmark3",
                    # `attr_x` represents the variables of the dataset you want to use to predict 
                    # the output.
                    attr_x=("prod_p", "prod_v", "load_p", "load_q"),
                    # `attr_y` represents the variables of the dataset you want to predict
                    # all the powerflow variables alongside the voltage angles
                    attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "load_v", "prod_q", "theta_or", "theta_ex"),
                    # `lr` is the learning rate
                    lr=3e-4, 
                    # `layer` is the type of keras layer you want to use. We don't recommend to 
                    # change it
                    layer=Dense,
                    # `layer_act` is the activation function you want to use after each layer
                    layer_act="relu",
                    # `loss` is the training loss
                    loss="mse",  # loss used to train the model
                    # `batch_size` is the size of the batch for training
                    batch_size=128,
                    # the method used to encode the topology vector
                    topo_vect_to_tau="all")

In [16]:
leapNet.train(nb_iter=200,
              train_dataset=neurips_benchmark3.train_dataset,
              val_dataset=neurips_benchmark3.val_dataset
              )

Epoch 1/200
782/782 [==============================] - 16s 12ms/step - loss: 16.2418 - a_or_hat_loss: 2.0659 - a_ex_hat_loss: 2.3618 - p_or_hat_loss: 2.2471 - p_ex_hat_loss: 2.1745 - q_ex_hat_loss: 3.2774 - q_or_hat_loss: 2.2160 - v_or_hat_loss: 0.2093 - v_ex_hat_loss: 0.2534 - load_v_hat_loss: 0.2285 - prod_q_hat_loss: 0.3535 - theta_or_hat_loss: 0.3973 - theta_ex_hat_loss: 0.4572 - val_loss: 0.6174 - val_a_or_hat_loss: 0.0914 - val_a_ex_hat_loss: 0.0916 - val_p_or_hat_loss: 0.0693 - val_p_ex_hat_loss: 0.0640 - val_q_ex_hat_loss: 0.1191 - val_q_or_hat_loss: 0.0857 - val_v_or_hat_loss: 0.0022 - val_v_ex_hat_loss: 0.0016 - val_load_v_hat_loss: 0.0077 - val_prod_q_hat_loss: 0.0228 - val_theta_or_hat_loss: 0.0291 - val_theta_ex_hat_loss: 0.0329
Epoch 2/200
782/782 [==============================] - 8s 10ms/step - loss: 0.4448 - a_or_hat_loss: 0.0651 - a_ex_hat_loss: 0.0679 - p_or_hat_loss: 0.0493 - p_ex_hat_loss: 0.0443 - q_ex_hat_loss: 0.0819 - q_or_hat_loss: 0.0596 - v_or_hat_loss: 0.00

782/782 [==============================] - 10s 13ms/step - loss: 0.0409 - a_or_hat_loss: 0.0063 - a_ex_hat_loss: 0.0062 - p_or_hat_loss: 0.0038 - p_ex_hat_loss: 0.0041 - q_ex_hat_loss: 0.0073 - q_or_hat_loss: 0.0057 - v_or_hat_loss: 7.4318e-05 - v_ex_hat_loss: 9.9531e-05 - load_v_hat_loss: 7.5809e-04 - prod_q_hat_loss: 0.0025 - theta_or_hat_loss: 0.0021 - theta_ex_hat_loss: 0.0021 - val_loss: 0.0455 - val_a_or_hat_loss: 0.0072 - val_a_ex_hat_loss: 0.0074 - val_p_or_hat_loss: 0.0041 - val_p_ex_hat_loss: 0.0043 - val_q_ex_hat_loss: 0.0083 - val_q_or_hat_loss: 0.0065 - val_v_or_hat_loss: 7.0889e-05 - val_v_ex_hat_loss: 8.0146e-05 - val_load_v_hat_loss: 6.9860e-04 - val_prod_q_hat_loss: 0.0024 - val_theta_or_hat_loss: 0.0022 - val_theta_ex_hat_loss: 0.0022
Epoch 12/200
782/782 [==============================] - 10s 13ms/step - loss: 0.0366 - a_or_hat_loss: 0.0057 - a_ex_hat_loss: 0.0057 - p_or_hat_loss: 0.0033 - p_ex_hat_loss: 0.0035 - q_ex_hat_loss: 0.0066 - q_or_hat_loss: 0.0051 - v_or_h

782/782 [==============================] - 9s 12ms/step - loss: 0.0213 - a_or_hat_loss: 0.0036 - a_ex_hat_loss: 0.0035 - p_or_hat_loss: 0.0017 - p_ex_hat_loss: 0.0018 - q_ex_hat_loss: 0.0038 - q_or_hat_loss: 0.0030 - v_or_hat_loss: 3.5190e-05 - v_ex_hat_loss: 5.3190e-05 - load_v_hat_loss: 3.8466e-04 - prod_q_hat_loss: 0.0012 - theta_or_hat_loss: 0.0011 - theta_ex_hat_loss: 0.0010 - val_loss: 0.0250 - val_a_or_hat_loss: 0.0041 - val_a_ex_hat_loss: 0.0044 - val_p_or_hat_loss: 0.0020 - val_p_ex_hat_loss: 0.0022 - val_q_ex_hat_loss: 0.0047 - val_q_or_hat_loss: 0.0035 - val_v_or_hat_loss: 2.7375e-05 - val_v_ex_hat_loss: 5.0216e-05 - val_load_v_hat_loss: 4.1909e-04 - val_prod_q_hat_loss: 0.0012 - val_theta_or_hat_loss: 0.0012 - val_theta_ex_hat_loss: 0.0012
Epoch 22/200
782/782 [==============================] - 9s 11ms/step - loss: 0.0209 - a_or_hat_loss: 0.0035 - a_ex_hat_loss: 0.0034 - p_or_hat_loss: 0.0016 - p_ex_hat_loss: 0.0018 - q_ex_hat_loss: 0.0038 - q_or_hat_loss: 0.0030 - v_or_hat

782/782 [==============================] - 9s 12ms/step - loss: 0.0152 - a_or_hat_loss: 0.0027 - a_ex_hat_loss: 0.0025 - p_or_hat_loss: 0.0011 - p_ex_hat_loss: 0.0012 - q_ex_hat_loss: 0.0027 - q_or_hat_loss: 0.0022 - v_or_hat_loss: 2.2738e-05 - v_ex_hat_loss: 3.8340e-05 - load_v_hat_loss: 2.9525e-04 - prod_q_hat_loss: 8.6586e-04 - theta_or_hat_loss: 8.1332e-04 - theta_ex_hat_loss: 7.6484e-04 - val_loss: 0.0175 - val_a_or_hat_loss: 0.0032 - val_a_ex_hat_loss: 0.0031 - val_p_or_hat_loss: 0.0013 - val_p_ex_hat_loss: 0.0013 - val_q_ex_hat_loss: 0.0030 - val_q_or_hat_loss: 0.0025 - val_v_or_hat_loss: 2.0757e-05 - val_v_ex_hat_loss: 3.6645e-05 - val_load_v_hat_loss: 3.1031e-04 - val_prod_q_hat_loss: 9.3260e-04 - val_theta_or_hat_loss: 9.7567e-04 - val_theta_ex_hat_loss: 8.2501e-04
Epoch 32/200
782/782 [==============================] - 8s 11ms/step - loss: 0.0145 - a_or_hat_loss: 0.0026 - a_ex_hat_loss: 0.0025 - p_or_hat_loss: 0.0011 - p_ex_hat_loss: 0.0011 - q_ex_hat_loss: 0.0025 - q_or_hat

782/782 [==============================] - 8s 11ms/step - loss: 0.0120 - a_or_hat_loss: 0.0022 - a_ex_hat_loss: 0.0021 - p_or_hat_loss: 9.0817e-04 - p_ex_hat_loss: 8.7936e-04 - q_ex_hat_loss: 0.0020 - q_or_hat_loss: 0.0017 - v_or_hat_loss: 1.5961e-05 - v_ex_hat_loss: 2.9457e-05 - load_v_hat_loss: 2.2329e-04 - prod_q_hat_loss: 6.6255e-04 - theta_or_hat_loss: 6.1687e-04 - theta_ex_hat_loss: 5.7969e-04 - val_loss: 0.0154 - val_a_or_hat_loss: 0.0029 - val_a_ex_hat_loss: 0.0030 - val_p_or_hat_loss: 0.0010 - val_p_ex_hat_loss: 0.0011 - val_q_ex_hat_loss: 0.0027 - val_q_or_hat_loss: 0.0021 - val_v_or_hat_loss: 1.9505e-05 - val_v_ex_hat_loss: 3.8479e-05 - val_load_v_hat_loss: 2.0291e-04 - val_prod_q_hat_loss: 8.3439e-04 - val_theta_or_hat_loss: 8.0765e-04 - val_theta_ex_hat_loss: 6.4955e-04
Epoch 42/200
782/782 [==============================] - 8s 10ms/step - loss: 0.0118 - a_or_hat_loss: 0.0022 - a_ex_hat_loss: 0.0021 - p_or_hat_loss: 8.9191e-04 - p_ex_hat_loss: 8.5096e-04 - q_ex_hat_loss: 0

...

782/782 [==============================] - 11s 14ms/step - loss: 0.0045 - a_or_hat_loss: 8.8253e-04 - a_ex_hat_loss: 9.5917e-04 - p_or_hat_loss: 3.1365e-04 - p_ex_hat_loss: 2.9758e-04 - q_ex_hat_loss: 7.2968e-04 - q_or_hat_loss: 6.4007e-04 - v_or_hat_loss: 4.9447e-06 - v_ex_hat_loss: 6.6492e-06 - load_v_hat_loss: 7.5794e-05 - prod_q_hat_loss: 2.1391e-04 - theta_or_hat_loss: 1.9443e-04 - theta_ex_hat_loss: 1.8573e-04 - val_loss: 0.0064 - val_a_or_hat_loss: 0.0012 - val_a_ex_hat_loss: 0.0013 - val_p_or_hat_loss: 4.9614e-04 - val_p_ex_hat_loss: 5.4464e-04 - val_q_ex_hat_loss: 9.5485e-04 - val_q_or_hat_loss: 9.8712e-04 - val_v_or_hat_loss: 3.2534e-06 - val_v_ex_hat_loss: 5.6763e-06 - val_load_v_hat_loss: 9.7198e-05 - val_prod_q_hat_loss: 2.6335e-04 - val_theta_or_hat_loss: 2.7498e-04 - val_theta_ex_hat_loss: 2.8485e-04
Epoch 158/200
782/782 [==============================] - 11s 14ms/step - loss: 0.0047 - a_or_hat_loss: 9.0390e-04 - a_ex_hat_loss: 9.8509e-04 - p_or_hat_loss: 3.4123e-04 - p

Epoch 167/200
782/782 [==============================] - 9s 12ms/step - loss: 0.0046 - a_or_hat_loss: 8.6855e-04 - a_ex_hat_loss: 9.6023e-04 - p_or_hat_loss: 3.3016e-04 - p_ex_hat_loss: 3.1400e-04 - q_ex_hat_loss: 7.3731e-04 - q_or_hat_loss: 6.5005e-04 - v_or_hat_loss: 5.9644e-06 - v_ex_hat_loss: 7.6227e-06 - load_v_hat_loss: 7.4173e-05 - prod_q_hat_loss: 2.2244e-04 - theta_or_hat_loss: 1.9362e-04 - theta_ex_hat_loss: 1.9259e-04 - val_loss: 0.0061 - val_a_or_hat_loss: 0.0011 - val_a_ex_hat_loss: 0.0013 - val_p_or_hat_loss: 4.8070e-04 - val_p_ex_hat_loss: 4.1515e-04 - val_q_ex_hat_loss: 0.0011 - val_q_or_hat_loss: 7.7489e-04 - val_v_or_hat_loss: 6.9593e-06 - val_v_ex_hat_loss: 9.4332e-06 - val_load_v_hat_loss: 9.3291e-05 - val_prod_q_hat_loss: 3.4175e-04 - val_theta_or_hat_loss: 2.4749e-04 - val_theta_ex_hat_loss: 2.5986e-04
Epoch 168/200
782/782 [==============================] - 9s 12ms/step - loss: 0.0043 - a_or_hat_loss: 8.4810e-04 - a_ex_hat_loss: 9.3434e-04 - p_or_hat_loss: 3.0506

Epoch 177/200
782/782 [==============================] - 10s 12ms/step - loss: 0.0044 - a_or_hat_loss: 8.3771e-04 - a_ex_hat_loss: 9.4108e-04 - p_or_hat_loss: 3.0907e-04 - p_ex_hat_loss: 2.9697e-04 - q_ex_hat_loss: 7.0439e-04 - q_or_hat_loss: 6.1533e-04 - v_or_hat_loss: 4.5722e-06 - v_ex_hat_loss: 6.0223e-06 - load_v_hat_loss: 8.3341e-05 - prod_q_hat_loss: 2.1090e-04 - theta_or_hat_loss: 1.8844e-04 - theta_ex_hat_loss: 1.8099e-04 - val_loss: 0.0058 - val_a_or_hat_loss: 0.0011 - val_a_ex_hat_loss: 0.0012 - val_p_or_hat_loss: 3.5429e-04 - val_p_ex_hat_loss: 3.3410e-04 - val_q_ex_hat_loss: 9.0294e-04 - val_q_or_hat_loss: 6.9226e-04 - val_v_or_hat_loss: 2.5521e-06 - val_v_ex_hat_loss: 5.0391e-06 - val_load_v_hat_loss: 1.2371e-04 - val_prod_q_hat_loss: 4.2429e-04 - val_theta_or_hat_loss: 3.2655e-04 - val_theta_ex_hat_loss: 3.4113e-04
Epoch 178/200
782/782 [==============================] - 9s 12ms/step - loss: 0.0042 - a_or_hat_loss: 8.2691e-04 - a_ex_hat_loss: 9.1302e-04 - p_or_hat_loss: 2

782/782 [==============================] - 10s 13ms/step - loss: 0.0040 - a_or_hat_loss: 7.8539e-04 - a_ex_hat_loss: 8.9041e-04 - p_or_hat_loss: 2.7677e-04 - p_ex_hat_loss: 2.6363e-04 - q_ex_hat_loss: 6.5075e-04 - q_or_hat_loss: 5.6152e-04 - v_or_hat_loss: 3.9117e-06 - v_ex_hat_loss: 5.3890e-06 - load_v_hat_loss: 6.3084e-05 - prod_q_hat_loss: 1.9569e-04 - theta_or_hat_loss: 1.6362e-04 - theta_ex_hat_loss: 1.6740e-04 - val_loss: 0.0063 - val_a_or_hat_loss: 0.0011 - val_a_ex_hat_loss: 0.0014 - val_p_or_hat_loss: 4.6322e-04 - val_p_ex_hat_loss: 4.4749e-04 - val_q_ex_hat_loss: 0.0012 - val_q_or_hat_loss: 9.4534e-04 - val_v_or_hat_loss: 7.0999e-06 - val_v_ex_hat_loss: 8.4003e-06 - val_load_v_hat_loss: 5.9413e-05 - val_prod_q_hat_loss: 2.2901e-04 - val_theta_or_hat_loss: 1.9877e-04 - val_theta_ex_hat_loss: 1.9801e-04
Epoch 187/200
782/782 [==============================] - 9s 12ms/step - loss: 0.0041 - a_or_hat_loss: 7.9321e-04 - a_ex_hat_loss: 9.0039e-04 - p_or_hat_loss: 2.9115e-04 - p_ex_h

Epoch 196/200
782/782 [==============================] - 9s 11ms/step - loss: 0.0040 - a_or_hat_loss: 7.7099e-04 - a_ex_hat_loss: 8.8339e-04 - p_or_hat_loss: 2.7493e-04 - p_ex_hat_loss: 2.6422e-04 - q_ex_hat_loss: 6.3030e-04 - q_or_hat_loss: 5.5412e-04 - v_or_hat_loss: 4.0005e-06 - v_ex_hat_loss: 5.4085e-06 - load_v_hat_loss: 6.4826e-05 - prod_q_hat_loss: 1.9266e-04 - theta_or_hat_loss: 1.5687e-04 - theta_ex_hat_loss: 1.6754e-04 - val_loss: 0.0050 - val_a_or_hat_loss: 9.7818e-04 - val_a_ex_hat_loss: 0.0011 - val_p_or_hat_loss: 3.8464e-04 - val_p_ex_hat_loss: 3.5477e-04 - val_q_ex_hat_loss: 7.5015e-04 - val_q_or_hat_loss: 6.2938e-04 - val_v_or_hat_loss: 2.3835e-06 - val_v_ex_hat_loss: 3.6596e-06 - val_load_v_hat_loss: 7.5233e-05 - val_prod_q_hat_loss: 3.0625e-04 - val_theta_or_hat_loss: 1.8245e-04 - val_theta_ex_hat_loss: 2.3072e-04
Epoch 197/200
782/782 [==============================] - 9s 12ms/step - loss: 0.0039 - a_or_hat_loss: 7.6797e-04 - a_ex_hat_loss: 8.6958e-04 - p_or_hat_loss

save it

In [12]:
leapNet.save(path_save)
leapNet.save_metadata(path_save)

load it

In [4]:
path_trained_models = os.path.join("trained_models")
from tensorflow.keras.layers import Dense
from lips.augmented_simulators import LeapNetAS

# recreate the baseline
leapNet = LeapNetAS(name="test_leapNetAS_benchmark3",
                    attr_x=("prod_p", "prod_v", "load_p", "load_q"),
                    attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "theta_or", "theta_ex"),
                    lr=3e-4, 
                    layer=Dense,
                    layer_act="relu",
                    loss="mse",  # loss used to train the model
                    batch_size=128,
                    topo_vect_to_tau="all")
# TODO create a wrapper for these 3 calls
leapNet.load_metadata(path_trained_models)
leapNet.init()
leapNet.restore(path_trained_models)

# Evaluate it 

In [17]:
leapNet_metrics_per_dataset = neurips_benchmark3.evaluate_augmented_simulator(leapNet)

A log file including some verifications is created at root directory with the name logs.log


C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add

In [19]:
neurips_benchmark3.predictions.keys()

dict_keys(['val', 'test', 'test_ood_topo'])

In [21]:
import numpy as np
np.save("predictions_LeapNet_test", neurips_benchmark3.predictions["test"])
np.save("predictions_LeapNet_test_ood", neurips_benchmark3.predictions["test_ood_topo"])

In [22]:
import pickle
a_file = open("ref_obs_ood.pkl", "wb")
pickle.dump(neurips_benchmark3._test_ood_topo_dataset.data, a_file)
a_file.close()

a_file = open("predictions_LeapNet_test.pkl", "wb")
pickle.dump(neurips_benchmark3.predictions["test"], a_file)
a_file.close()

a_file = open("predictions_LeapNet_test_ood.pkl", "wb")
pickle.dump(neurips_benchmark3.predictions["test_ood_topo"], a_file)
a_file.close()

### ML Metrics 

In [14]:
ML_metrics = 0
print("MAPE90")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["mape90"])
print("MAPE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["mape_avg"])
print("MAE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.007627280422307372,
 'a_or': 0.008161371365289707,
 'p_ex': 0.0070716083679348615,
 'p_or': 0.006713402995507482,
 'q_ex': 0.01872287409967809,
 'q_or': 0.026856733574902935,
 'theta_ex': 0.008855646218386186,
 'theta_or': nan,
 'v_ex': 0.0027800687775338343,
 'v_or': 0.0027032407203839343}
MAPE
{'a_ex': 0.019798308458008926,
 'a_or': 0.020379822565709783,
 'p_ex': 0.01460759526570545,
 'p_or': 0.013359139042767925,
 'q_ex': 0.02137782497860824,
 'q_or': 0.021455024094834484,
 'theta_ex': 0.008508733215873797,
 'theta_or': nan,
 'v_ex': 0.0024370175344503144,
 'v_or': 0.0023524734635076232}
MAE
{'a_ex': 3.9130992889404297,
 'a_or': 2.7250747680664062,
 'p_ex': 0.16841043531894684,
 'p_or': 0.15682396292686462,
 'q_ex': 0.10360975563526154,
 'q_or': 0.1071694865822792,
 'theta_ex': 0.0641272084852525,
 'theta_or': 0.05289448516613774,
 'v_ex': 0.14341893792152405,
 'v_or': 0.1785714030265808}
NRMSE
{'a_ex': 0.004338228143751621,
 'a_or': 0.004624119959771633,
 'p_ex': 

### OOD Generalization

In [15]:
print("MAPE90")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape90"])
print("MAPE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape_avg"])
print("MAE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.13353149307680554,
 'a_or': 0.1362641806290948,
 'p_ex': 0.1498188134163031,
 'p_or': 0.13974530298697216,
 'q_ex': 0.23166653743605742,
 'q_or': 0.26613770426753536,
 'theta_ex': 0.10928676934889206,
 'theta_or': nan,
 'v_ex': 0.016116134482477444,
 'v_or': 0.015510716400258287}
MAPE
{'a_ex': 0.16660052068382186,
 'a_or': 0.16457392740735016,
 'p_ex': 0.17867681365282212,
 'p_or': 0.16137577897119917,
 'q_ex': 0.18446503347992374,
 'q_or': 0.1857152761019446,
 'theta_ex': 0.09906898033527073,
 'theta_or': nan,
 'v_ex': 0.010273143069932756,
 'v_or': 0.010370839340545229}
MAE
{'a_ex': 44.475318908691406,
 'a_or': 31.157516479492188,
 'p_ex': 2.0687756538391113,
 'p_or': 1.895237684249878,
 'q_ex': 0.9647682309150696,
 'q_or': 0.9432681202888489,
 'theta_ex': 1.3143264040202853,
 'theta_or': 1.2318205872948247,
 'v_ex': 0.5145779848098755,
 'v_or': 0.676001787185669}
NRMSE
{'a_ex': 0.04387463629245758,
 'a_or': 0.042918186634778976,
 'p_ex': 0.032695792615413666,
 'p_o

## Physics compliances

In [18]:
PhysicCompliances = 1
print("current pos")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
#pprint(leapNet_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
#pprint(leapNet_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["line_status"])

current pos
0.000495
0.0003
voltage pos
loss
0.274485
line_status
{'a_ex_not_null': 0.0,
 'a_or_not_null': 0.0,
 'a_violations': 0.0,
 'p_ex_not_null': 0.0,
 'p_or_not_null': 0,
 'p_violations': 0.0,
 'q_ex_not_null': 0.0,
 'q_or_not_null': 0.0,
 'q_violations': 0.0}


In [1]:
leapNet_metrics_per_dataset["test"][1]["KCL_new"]

NameError: name 'leapNet_metrics_per_dataset' is not defined

In [19]:
PhysicCompliances = 1
print("current pos")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
#pprint(leapNet_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
#pprint(leapNet_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["line_status"])
#pprint(dc_metrics_per_dataset["test_ood_topo"][1])

current pos
0.00393
0.004095
voltage pos
loss
0.39763
line_status
{'a_ex_not_null': 0.0,
 'a_or_not_null': 0.0,
 'a_violations': 0.0,
 'p_ex_not_null': 0.0,
 'p_or_not_null': 0,
 'p_violations': 0.0,
 'q_ex_not_null': 0.0,
 'q_or_not_null': 0.0,
 'q_violations': 0.0}
